In [ ]:
import numpy as np
import PIL
import random
from IPython.core.display import Image, display

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.autograd import Function
from torch.autograd import gradcheck

import torchvision
import torchvision.transforms as transforms
import torch.utils.data as data

import math

import matplotlib.pyplot as plt
%matplotlib inline

torch.manual_seed(2)
np.random.seed(2)

# правильно выбирайте карту, иначе все упадет!
CUDA_DEVICE=0

def data2image(data):
    res = np.transpose(np.reshape(data ,(3, 32,32)), (1,2,0))
    return PIL.Image.fromarray(np.uint8(res))

def imshow(img):
    npimg = img.numpy().astype('uint8')
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [ ]:
class CustomC100Dataset(data.Dataset):
    def __init__(self, dataset_path, dataset_type, img_cnt = 50000, transform = None, target_transform = None):
        if dataset_type not in ['train', 'test']:
            raise "Unknown dataset type : {}".format(dataset_type)
        self.ds_type = dataset_type
        self.ds_path = dataset_path
        self.img_cnt = img_cnt
        self.transform = transform
        self.t_transform = target_transform
        self.__load__()
    
    def __load__(self):
        dataset = np.load(self.ds_path)
        if self.ds_type == 'train':
            dataset = dataset.reshape((self.img_cnt,3073))
            self.y, self.x = np.hsplit(dataset, [1])
            self.y = self.y.astype(np.int64)
            self.x = self.x.reshape((self.x.shape[0],3,32,32))
            self.x = self.x.transpose((0, 2, 3, 1))
        
        if self.ds_type == 'test':
            dataset = dataset.reshape((self.img_cnt,3072))
            self.y = np.zeros((dataset.shape[0], 1), dtype=np.int64)
            self.x = dataset.reshape((dataset.shape[0],3,32,32))
            self.x = self.x.transpose((0, 2, 3, 1))
    
    def __getitem__(self, index):
        img, target = self.x[index], self.y[index]
        
        img = PIL.Image.fromarray(img)
        
        if self.transform is not None:
            img = self.transform(img)

        if self.t_transform is not None:
            target = self.t_transform(target)
            
        return img, target

    
    def __len__(self):
        return len(self.x)

In [ ]:
transform_train = transforms.Compose([
    # тут надо сделать аугментации и преобразования для trainset
])

transform_test = transforms.Compose([
    # тут так же преобразования, но для testset
])

# подумайте, должны ли преобразования train и test сет быть абсолютно одинаковыми ?

train_dataset = CustomC100Dataset('homework_4.train.npy', 'train', img_cnt = 50000, transform = transform_train)
test_nc_dataset = CustomC100Dataset('homework_4_no_classes.test.npy', 'test', img_cnt = 10000, transform = transform_test)

In [ ]:
img, label = test_dataset[4]
imshow(img)

In [ ]:
class BasicBlock(nn.Module):
    expansion=1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        # конструктор базового блока
        
    def forward(self, x):
        # форвард базового блока

In [ ]:
# чтобы вы не запутались, сборку самой модели предоставляем
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=100):
        
        super(ResNet, self).__init__()
        self.inplanes = 16
        self.conv1 = # вставить сюда стартовую конволюцию из статьи
        self.bn1 = # bn-слой
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 16, layers[0])
        self.layer2 = self._make_layer(block, 32, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 64, layers[2], stride=2)
        self.avgpool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # написать инициализацию параметров конволюции
            elif isinstance(m, nn.BatchNorm2d):
                # инициализация параметров батч-нормализаций

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                # здесь необходимо привести слои для downsampling в shotrcut соединениях
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [ ]:
# Выбирайте любой ResNet

def ResNet20():
    return ResNet(BasicBlock, [3,3,3], 100)

def ResNet32():
    return ResNet(BasicBlock, [5,5,5], 100)

def ResNet44():
    return ResNet(BasicBlock, [7,7,7], 100)

def ResNet56():
    return ResNet(BasicBlock, [9,9,9], 100)

In [ ]:
def acc_check(model, validation_loader, batch_size):
    model.eval()
    correct = 0.0
    total = 0.0
    
    for batch_idx, (inputs, labels) in enumerate(validation_loader):
        inputs, labels = inputs.cuda(), labels.view(batch_size).cuda()
        inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)
        outputs = model(inputs)
        outputs = F.softmax(outputs)
        prob, predicted = torch.topk(outputs.data, 1)
        total += labels.size(0)
        correct += predicted.view(-1).eq(labels.data).sum()
        
    model.train()
    return (correct / total)

In [ ]:
def make_solution_pytorch(model, test_data_loader, batch_size):
    res = []
    model.eval()
    for batch_idx, (inputs, labels) in enumerate(test_data_loader):
        inputs, labels = inputs.cuda(), labels.view(batch_size).cuda()
        inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)
        outputs = model(inputs)
        outputs = F.softmax(outputs)
        prob, predicted = torch.topk(outputs.data, 1)
        res = np.append(res, predicted.view(-1).cpu().numpy())
    model.train()
    return res

In [ ]:
model = ResNetXX()
model.cuda(CUDA_DEVICE)

#all parameters from https://arxiv.org/pdf/1512.03385.pdf
epochs_cnt = # кол-во эпох

# эти параметры тоже на ваше усмотрение
minibatch_size = 125 
test_batch_size = 100

optimizer = # выбираем отимизатор 

criterion = nn.CrossEntropyLoss().cuda(CUDA_DEVICE)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = minibatch_size, shuffle = True, num_workers = 2)
test_nc_loader = torch.utils.data.DataLoader(test_nc_dataset, batch_size = test_batch_size, shuffle = False, num_workers = 2)
#losses = []

# примерный код для обучения модели, не обязательно строго следовать ему
for epoch in range(epochs_cnt):    
    train_loss = 0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        if(batch_idx % 10 == 0):
            print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, train_loss / 10))
            train_loss = 0
        inputs, labels = inputs.cuda(), labels.view(minibatch_size).cuda()
        optimizer.zero_grad()
        inputs, labels = Variable(inputs), Variable(labels)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        train_loss += loss.data[0]
        loss.backward()
        optimizer.step()

print('Finished Training')

In [ ]:
# генерим решение вашей модели
solution = make_solution_pytorch(model, test_nc_loader, test_batch_size)
with open('my_solution.csv', 'w') as fout:
    print('Id', 'Prediction', sep=',', file=fout)
    for i, prediction in enumerate(solution):
        print(i, int(prediction), sep=',', file=fout)